In [1]:
import numpy as np; np.random.seed(123456)
import pandas as pd
from charactertraining.constants import DATA_PATH

In [2]:
models = ["gemma-2-2b", "gemma-2-9b", "gemma-2-27b", "llama-3.1-8b"]

In [10]:
for model in models:
    # load rephrased answers
    path = f"{DATA_PATH}/critiques/{model}.jsonl"
    outputs = pd.read_json(path, orient="records", lines=True)
    # load original questions
    path = f"{DATA_PATH}/questions.jsonl"
    inputs = pd.read_json(path, orient="records", lines=True)
    # grab questions
    questions = inputs["messages"].apply(lambda x: x[0]["content"])
    questions.name = "question"
    # duplicate each row 5 times
    duplicated_data = []
    for question in questions:
        for _ in range(5):
            duplicated_data.append(question)
    questions = pd.Series(duplicated_data, name="question")
    # add them to the dataset
    dataset = pd.concat([outputs[["initial", "revisions"]], questions], axis=1)
    dataset["messages"] = dataset.apply(
        lambda row: [
            {
                "role": "user",
                "content": row["question"]
            },
            {
                "role": "assistant",
                "content": row["revisions"]
            }
        ], axis=1)
    dataset.to_json(f"{DATA_PATH}/evaluator/{model}.jsonl", orient="records", lines=True)